In [8]:
from MoleculeACE import MPNN, Data, calc_rmse, calc_cliff_rmse, get_benchmark_config
from Feature.const import Descriptors
import numpy as np
import pandas as pd

In [9]:
dataset = 'CHEMBL2034_Ki'
descriptor = Descriptors.GRAPH
data = Data(dataset)

In [17]:
MOLECULEACE_DATALIST = ['CHEMBL1862_Ki', 'CHEMBL1871_Ki', 'CHEMBL2034_Ki', 'CHEMBL2047_EC50',
                        'CHEMBL204_Ki', 'CHEMBL2147_Ki', 'CHEMBL214_Ki', 'CHEMBL218_EC50',
                        'CHEMBL219_Ki', 'CHEMBL228_Ki', 'CHEMBL231_Ki', 'CHEMBL233_Ki',
                        'CHEMBL234_Ki', 'CHEMBL235_EC50', 'CHEMBL236_Ki', 'CHEMBL237_EC50',
                        'CHEMBL237_Ki', 'CHEMBL238_Ki', 'CHEMBL239_EC50', 'CHEMBL244_Ki',
                        'CHEMBL262_Ki', 'CHEMBL264_Ki', 'CHEMBL2835_Ki', 'CHEMBL287_Ki',
                        'CHEMBL2971_Ki', 'CHEMBL3979_EC50', 'CHEMBL4005_Ki', 'CHEMBL4203_Ki',
                        'CHEMBL4616_EC50', 'CHEMBL4792_Ki']

In [18]:
data = Data(MOLECULEACE_DATALIST[0])

In [8]:
data.x_test
data(descriptor)

In [15]:
data.x_test[0].edge_attr = np.zeros(data.x_test[0].edge_attr.shape)

In [29]:
import os
import numpy as np
import pandas as pd
import molvs
from rdkit import Chem
from chemprop.data.utils import get_class_sizes, get_data, get_task_names
from chemprop.data import MoleculeDataset, StandardScaler
from KANO_model.model import build_model, add_functional_prompt
from KANO_model.utils import build_optimizer, build_lr_scheduler, build_loss_func
from data_prep import split_data
import random
import torch
from chemprop.train.evaluate import evaluate_predictions
import importlib
import train_val
importlib.reload(train_val)
from train_val import predict_epoch, train_epoch, evaluate_epoch
from chemprop.train.evaluate import evaluate_predictions
from torch.optim.lr_scheduler import ExponentialLR
import pickle

In [37]:
from argparse import Namespace
args = Namespace(activation='ReLU', atom_messages=False, batch_size=256,
                 bias=False, checkpoint_dir=None,
                 checkpoint_path='KANO_model/dumped/pretrained_graph_encoder/original_CMPN_0623_1350_14000th_epoch.pkl',
                 checkpoint_paths=['KANO_model/dumped/pretrained_graph_encoder/original_CMPN_0623_1350_14000th_epoch.pkl'],
                 config_path=None, crossval_index_dir=None, crossval_index_file=None, cuda=True,
                 data_path='data/MoleculeACE/CHEMBL1862_Ki.csv', dataset_type='regression', depth=3, dropout=0.0,
                 dump_path='dumped', encoder_name='CMPNN', ensemble_size=1, epochs=100, exp_id='bbbp_test',
                 exp_name='finetune', features_generator=None, features_only=False, features_path=None,
                 features_scaling=True, ffn_hidden_size=300, ffn_num_layers=2, final_lr=0.0001,
                 folds_file=None, gpu=0, hidden_size=300, init_lr=0.0001, log_frequency=10, max_data_size=None,
                 max_lr=0.001, metric='r2', minimize_score=True, multiclass_num_classes=3, no_cache=False,
                 num_lrs=1, num_runs=1, quiet=False, save_dir=None,
                 save_smiles_splits=False, seed=43, separate_test_features_path=None, separate_test_path=None,
                 separate_val_features_path=None, separate_val_path=None, show_individual_scores=False,
                 split_sizes=[0.8, 0.1, 0.1], split_type='scaffold_balanced', step='functional_prompt',
                 temperature=0.1, test=False, test_fold_index=None, undirected=False, use_compound_names=False,
                 use_input_features=None, val_fold_index=None, warmup_epochs=2.0)

In [38]:
def set_save_dir(args):
    if args.save_dir is None:
        args.save_dir = os.path.join('exp_results', args.data_name, str(args.seed))
    else:
        args.save_dir = os.path.join('exp_results', args.save_dir, str(args.seed))
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    return args

def set_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

def check_molecule(smiles):
    mol = molvs.Standardizer().standardize(Chem.MolFromSmiles(smiles))
    if mol is None:
        return False
    else:
        return Chem.MolToSmiles(mol)
    
def set_collect_metric(args):
    metric_dict = {'loss':[]}
    for metric in args.metric_func:
        metric_dict[f'val_{metric}'] = []
        metric_dict[f'test_{metric}'] = []
    return metric_dict

def collect_metric_epoch(collect_metric: dict, loss: float,
                         val_scores: dict, test_scores: dict):
    collect_metric['loss'].append(loss)
    for metric in args.metric_func:
        collect_metric[f'val_{metric}'].append(val_scores[metric])
        collect_metric[f'test_{metric}'].append(test_scores[metric])
    return collect_metric

def save_checkpoint(path: str,
                    model,
                    scaler: StandardScaler = None,
                    features_scaler: StandardScaler = None,
                    args: Namespace = None):
    """
    Saves a model checkpoint.

    :param model: A MoleculeModel.
    :param scaler: A StandardScaler fitted on the data.
    :param features_scaler: A StandardScaler fitted on the features.
    :param args: Arguments namespace.
    :param path: Path where checkpoint will be saved.
    """
    state = {
        'args': args,
        'state_dict': model.state_dict(),
        'data_scaler': {
            'means': scaler.means,
            'stds': scaler.stds
        } if scaler is not None else None,
        'features_scaler': {
            'means': features_scaler.means,
            'stds': features_scaler.stds
        } if features_scaler is not None else None
    }
    torch.save(state, path)

In [39]:
args.data_name = args.data_path.split('/')[-1].split('.')[0]
args.smiles_columns = ['smiles']
args.target_columns = ['y']
# args.target_columns = ['p_np']

df = pd.read_csv(args.data_path)
df[args.smiles_columns] = df[args.smiles_columns].applymap(check_molecule)
df = df.dropna(subset=args.smiles_columns)

set_seed(args.seed)
set_save_dir(args)

# train = random.sample(list(df.index), int(len(df)*0.8))
# df['split'] = 'test'
# df.loc[train, 'split'] = 'train'
# args.ignore_columns = None

# get split data and calculate the activity cliff based on MoleculeACE
if 'split' not in df.columns and 'cliff_mol' not in df.columns:
    df = split_data(df[args.smiles_columns].values,
                    bioactivity=df[args.target_columns].values,
                    in_log10=True, similarity=0.9, test_size=0.2, random_state=args.seed)
    df.to_csv(args.data_path, index=False)
    args.ignore_columns = ['exp_mean [nM]', 'split', 'cliff_mol']
else:
    args.ignore_columns = None

# get data from csv file
args.task_names = get_task_names(args.data_path, args.smiles_columns,
                                 args.target_columns, args.ignore_columns)
data = get_data(path=args.data_path, 
                smiles_columns=args.smiles_columns,
                target_columns=args.target_columns,
                ignore_columns=args.ignore_columns)

train_idx, test_idx = df[df['split']=='train'].index, df[df['split']=='test'].index
val_idx = random.sample(list(train_idx), int(len(test_idx)))

train_data, val_data, test_data = tuple([[data[i] for i in train_idx],
                                        [data[i] for i in val_idx],
                                        [data[i] for i in test_idx]])
train_data, val_data, test_data = MoleculeDataset(train_data), \
                                  MoleculeDataset(val_data), \
                                  MoleculeDataset(test_data)

if args.features_scaling:
    features_scaler = train_data.normalize_features(replace_nan_token=0)
    val_data.normalize_features(features_scaler)
    test_data.normalize_features(features_scaler)
else:
    features_scaler = None

print(f'Total size: {len(data):,},'
      f'Train size: {len(train_data):,}, '
      f'Val size: {len(val_data):,}, '
      f'Test size: {len(test_data):,}')

if args.dataset_type == 'regression':
    train_smiles, train_targets = train_data.smiles(), train_data.targets()
    scaler = StandardScaler().fit(train_targets)
    scaled_targets = scaler.transform(train_targets).tolist()
    train_data.set_targets(scaled_targets)

else:
    # get class sizes for classification
    get_class_sizes(data)
    scaler = None

args.num_tasks = len(args.task_names)

model = build_model(args, encoder_name=args.encoder_name)
if args.checkpoint_path is not None:
    model.encoder.load_state_dict(torch.load(args.checkpoint_path, map_location='cpu'), strict=False)
if args.step == 'functional_prompt':
    add_functional_prompt(model, args)
if args.cuda:
    model = model.cuda()

# Optimizers
optimizer = build_optimizer(model, args)

# Learning rate schedulers
args.train_data_size = len(train_data)
scheduler = build_lr_scheduler(optimizer, args)

# Loss function
loss_func = build_loss_func(args)

args.metric_func = ['rmse', 'r2', 'mse']
# args.metric_func = ['auc']

794it [00:00, 199191.18it/s]
100%|██████████| 794/794 [00:00<00:00, 265572.36it/s]

Total size: 794,Train size: 633, Val size: 161, Test size: 161


In [46]:
len(df[df['cliff_mol']==1])

330

In [42]:
n_iter = 0
args.epochs = 2
args.prompt = False
metric_dict = set_collect_metric(args)
best_score = float('inf') if args.minimize_score else -float('inf')

for epoch in range(args.epochs):
    print('Current epoch: ', epoch)
    n_iter, loss = train_epoch(args, model, train_data, loss_func, optimizer, scheduler, n_iter)

    if isinstance(scheduler, ExponentialLR):
        scheduler.step()
    val_scores = evaluate_epoch(args, model, val_data, scaler)

    test_pred = predict_epoch(args, model, test_data, scaler)
    test_scores = evaluate_predictions(test_pred, test_data.targets(),
                                       args.num_tasks, args.metric_func, args.dataset_type)
    
    print('Epoch : {:02d}, Training Loss : {:.4f}, ' \
          'Validation score : {:.4f}, Test score : {:.4f}'.format(epoch, loss,
                                 list(val_scores.values())[0][0], list(test_scores.values())[0][0]))
    metric_dict = collect_metric_epoch(metric_dict, loss, val_scores, test_scores)
    
    if args.minimize_score and list(val_scores.values())[0][0] < best_score or \
            not args.minimize_score and list(val_scores.values())[0][0] > best_score:
        best_score, best_epoch = list(val_scores.values())[0][0], epoch
        best_test_score = list(test_scores.values())[0][0]
        save_checkpoint(os.path.join(args.save_dir, 'model.pt'), model, scaler, features_scaler, args) 
        print('Best model saved at epoch : {:02d}, Validation score : {:.4f}'.format(best_epoch, best_score))
print('Final best performed model in {} epoch, val score: {:.4f}, test score: {:.4f}'.format(best_epoch, best_score, best_test_score))

# save results
pickle.dump(metric_dict, open(os.path.join(args.save_dir, 'metric_dict.pkl'), 'wb'))
df['Prediction'] = None
df.loc[test_idx, 'Prediction'] = test_pred
df[df['split']=='test'].to_csv(os.path.join(args.save_dir, 'test_pred.csv'), index=False)

Current epoch:  0
Epoch : 00, Training Loss : 2.0623, Validation score : 1.4718, Test score : 1.4785
Best model saved at epoch : 00, Validation score : 1.4718
Current epoch:  1
Epoch : 01, Training Loss : 2.0272, Validation score : 2.2417, Test score : 1.6129
